<a href="https://colab.research.google.com/github/yeonghun00/stock/blob/master/crawler/naverfinance_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# code: LG electronics
code = '066570' 

In [2]:
# avoid ban
# http://www.useragentstring.com/
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

In [3]:
# crawling page
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

finished = False
def page_to_df(page):
  url = "https://finance.naver.com/item/sise_day.nhn?code=" + code + "&page=" + str(page)
  result = requests.get(url, headers = headers)
  bs_obj = BeautifulSoup(result.content, "html.parser")
  tr = bs_obj.find_all("tr", {'onmouseover':'mouseOver(this)', 'onmouseout':"mouseOut(this)"})
  data_dict = {}
  for i in range(10):
    try: 
      spans = tr[i].find_all('span')
      lines = [span.get_text() for span in spans]
      objdate = datetime.datetime.strptime(lines[0], '%Y.%m.%d')
      data_dict[objdate] = lines[1:]
    except:
      break

  df = pd.DataFrame.from_dict(data_dict).transpose()
  df.columns = ['Close','_','Open',
                      'High','Low','Volume']
  df = df.drop(columns="_")

  return df

In [4]:
df = page_to_df(1)
for i in range(2, 9999):
  # 끝나면 끝내 (페이지번호 관계 없이 마지막 페이지 호출함 --> 앞과 비교)
  try:
    if (any(page_to_df(i).index != page_to_df(i-1).index)):
      df = df.append(page_to_df(i))
      ##########################
      print('.', end = '')
      if (i%100==0):
        print(i)
    else:
      break
  except:
    df = df.append(page_to_df(i))
    break


df = df.sort_index(axis = 0) 

...................................................................................................100
....................................................................................................200
....................................................................................................300
....................................................................................................400
........................................................

In [5]:
df

,Close,Open,High,Low,Volume
2002-04-23,"58,800","62,100","62,800","58,500","5,384,435"
2002-04-24,"57,000","59,500","60,800","54,300","4,659,371"
2002-04-25,"52,400","55,000","56,400","51,800","3,913,730"
2002-04-26,"54,800","51,800","56,000","50,300","4,176,615"
2002-04-29,"52,200","51,300","53,400","51,300","1,990,302"
...,...,...,...,...,...
2020-09-21,"90,900","90,100","92,500","89,400","1,050,515"
2020-09-22,"88,900","90,400","91,400","88,000","1,054,942"
2020-09-23,"91,200","89,500","91,500","88,000","1,121,711"
2020-09-24,"89,500","89,600","91,500","89,500","1,150,871"


In [6]:
from google.colab import files

file_name = code + '.csv'
df.to_csv(file_name)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>